# Q3: How Robust Credit Risk Models are Over Time

This script compares [INSERT MODEL NAMES] in their performance in credit risk prediction when trained and tested data differ in economic periods.

### Imports

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import subprocess
from pathlib import Path

# Add project root to path
project_root = Path().resolve().parent
sys.path.append(str(project_root))

from data_processings.datasets import LendingClubDataset

### Pre-Processing of Data

In [2]:
dataloader = LendingClubDataset()
accepted_df = dataloader.load(5000)

In [3]:
print(list(accepted_df.columns))

['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq',

In [4]:
accepted_df["loan_status"].unique()

array(['Fully Paid', 'Current', 'Charged Off', 'In Grace Period',
       'Late (31-120 days)', 'Late (16-30 days)'], dtype=object)

In [5]:
accepted_df[["dti", "dti_joint"]]

,dti,dti_joint
0,5.91,NaN
1,16.06,NaN
2,10.78,13.85
3,17.06,NaN
4,25.37,NaN
...,...,...
4995,31.88,NaN
4996,17.63,NaN
4997,35.01,NaN
4998,23.13,NaN


In [6]:
accepted_df['dti_final'] = accepted_df['dti_joint'].fillna(accepted_df['dti'])

In [9]:
accepted_df["term"].unique()


array([' 36 months', ' 60 months'], dtype=object)